now let's try 100 epochs

In [ ]:
%%capture output
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, BatchNormalization
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix
import pickle
import matplotlib.pyplot as plt
import os
from keras.applications import ResNet50



# Paths to training and testing data
training_data_path = '/home/wangg/REU-Hearing-Loss-Project-1/machine learning/preTrainedModel/data/10folds - 70-15-15 train test split (includes all subjects)/fold1/Training'
testing_data_path = '/home/wangg/REU-Hearing-Loss-Project-1/machine learning/preTrainedModel/data/10folds - 70-15-15 train test split (includes all subjects)/fold1/Testing'

print("WORKING NOW")
# Image dimensions
img_height, img_width = 224, 224

# Define batch size
batch_size = 32

# Create data generators for training and testing with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2
)
# Create data generators for training and testing
# train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    training_data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'  # bc we have two classes
)

# # Print the class names
# class_names = list(train_generator.class_indices.keys())
# print("Class Names:", class_names)

test_generator = test_datagen.flow_from_directory(
    testing_data_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

# Load the pre-trained ResNet-50 model with weights from ImageNet
base_model = ResNet50(
    include_top=False, # don't want to include the top layer because we will define our own input/output layer
    weights='imagenet',
    input_tensor=None,
    input_shape=(img_height, img_width, 3),
    # pooling=None,
    pooling= 'max',
    classes=2,
)
# Freeze only the convolutional layers of the pre-trained model
for layer in base_model.layers:
    if isinstance(layer, keras.layers.Conv2D):
        layer.trainable = False

    
# Add custom classification layers on top of ResNet-50
# these new layers and specifically the 512 neurons are the ones that will be learning the new weights 
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(2, activation='softmax')(x)  # Assuming 2 classes: Healthy and Hearing Impaired


# Create the final model
model = keras.Model(base_model.input, x)

# Print model summary
model.summary()

# # Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Count the number of batches per epoch
num_batches_per_epoch = len(train_generator)

# Calculate the total number of images seen during training
total_images_seen = batch_size * num_batches_per_epoch * 10  # Assuming 10 epochs

print(f"Total number of batches per epoch: {num_batches_per_epoch}")
print(f"Total number of images seen during training: {total_images_seen}")


# Train the model
history = model.fit(train_generator, epochs=100)

# Evaluate the model on the testing data
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

# Save accuracies during training
train_accuracies = history.history['accuracy']

# Evaluate the model on the testing data and get predictions
predictions = model.predict(test_generator)
predicted_labels = np.argmax(predictions, axis=1)

# Get true labels
true_labels = test_generator.classes

# Calculate precision, recall, and f1-score
precision, recall, f1_score, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='weighted')

# Save classification results to a file
results_filename = 'classification_results_pretrained_model_freeze_convo_layers_100epoch.txt'
with open(results_filename, 'w') as results_file:
    results_file.write("Image Name\tTrue Label\tPredicted Label\n")
    for i in range(len(test_generator.filenames)):
        image_name = os.path.basename(test_generator.filenames[i])
        true_label = true_labels[i]
        predicted_label = predicted_labels[i]
        results_file.write(f"{image_name}\t{true_label}\t{predicted_label}\n")

# Save metrics to a text file with a different filename
metrics_filename = 'classification_metrics_pretrained_model_freeze_convo_layers_100epoch.txt'
with open(metrics_filename, 'w') as file:
    file.write(f'Test Accuracy: {test_acc}\n')
    file.write(f'Precision: {precision}\n')
    file.write(f'Recall: {recall}\n')
    file.write(f'F1 Score: {f1_score}\n')

# Print confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)
print('Confusion Matrix:')
print(conf_matrix)